In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
word_to_ix = {'hello': 0, 'world': 1}
embeds = nn.Embedding(2, 5)

In [6]:
lookup_tensor = torch.LongTensor([word_to_ix['hello']])
hello_embed = embeds(autograd.Variable(lookup_tensor))

In [12]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [14]:
trigrams = [([test_sentence[i],test_sentence[i+1]], test_sentence[i+2]) for i in range(len(test_sentence) - 2)]

In [17]:
vocab = set(test_sentence)

In [20]:
word_to_ix = {word:i for i, word in enumerate(vocab)}

In [26]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(1, -1)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs

In [27]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [32]:
for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))
        model.zero_grad()
        log_probs = model(context_var)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))
        loss.backward()
        optimizer.step()
        total_loss += loss.data
    losses.append(total_loss)

In [33]:
losses

[
  523.1124
 [torch.FloatTensor of size 1], 
  520.3711
 [torch.FloatTensor of size 1], 
  517.6536
 [torch.FloatTensor of size 1], 
  514.9601
 [torch.FloatTensor of size 1], 
  512.2888
 [torch.FloatTensor of size 1], 
  509.6395
 [torch.FloatTensor of size 1], 
  507.0107
 [torch.FloatTensor of size 1], 
  504.3997
 [torch.FloatTensor of size 1], 
  501.8091
 [torch.FloatTensor of size 1], 
  499.2358
 [torch.FloatTensor of size 1], 
  496.6790
 [torch.FloatTensor of size 1], 
  494.1385
 [torch.FloatTensor of size 1], 
  491.6112
 [torch.FloatTensor of size 1], 
  489.0970
 [torch.FloatTensor of size 1], 
  486.5953
 [torch.FloatTensor of size 1], 
  484.1063
 [torch.FloatTensor of size 1], 
  481.6279
 [torch.FloatTensor of size 1], 
  479.1617
 [torch.FloatTensor of size 1], 
  476.7037
 [torch.FloatTensor of size 1], 
  474.2552
 [torch.FloatTensor of size 1]]